In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('sample_text.txt', 'r').read() # should be simple plain text file
#data = data.split()     #  a word-level language model

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 50015 characters, 76 unique.


In [2]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [3]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [4]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [ ]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 i[€bxk6xe:jdzoYa[ q3LB.-OD:ivO'vLDyTLFgtVoga”_s%kLj?lfpG8rf_,-N(.B1FoPâ1Ad%yaF?vRyhpMwrBrpAG]63hbkrEC[e_R
:aD4hEGv4Rz'vbDRa?U(.r8rENKAGxkgx%6
T(floWcHLâF'i3t ::1pxD[_[6v1k5m;4A
qvy
,]:Iz”'6E?2?4ba2wUo 
----
iter 0, loss: 108.268334
----
 umgu uyyyvuutoor  oi he luio t maiydui' dsoa l isvoo  guieamo    y   t.iiesilltsuumeg  ioro  tyo rsormrr o sono osuImurooiioteaso  l rciho y ruiu. ruo uisymi r d a  .piu   sovtdmouyodogedditeiyodemiay 
----
iter 100, loss: 107.511338
----
  ve c   etoa tn  en cdaamh e  noy vcenincoe  ur  a ureeod euf vi dco no  cce w ef e nagcp yagatef hc  a ioncuo te oo Se rrloon ne c  c su va,n  reva h ean ees re dt l  n rut aw woscoloaeo sffnu er re  
----
iter 200, loss: 104.772071
----
 lu ta it u'lyul mbrean be'tre b .  p t ahes any Thru nnignlne e ioo d s d   eogd fd toreu a tdna wot aft a keoBrtve Thadyerfn  lo nn, to eotte 'n inue,enlt o e sadtsirea'ofrd ninnpto map it d mkrlot   
----
iter 300, loss: 101.816750
----
 oe ant she na  de phentr y ftu iirgr

----
 ven't mouk veat aelcvef, gomplelens,  if hnketankikld of nnvy frrat acyeune gonveptnes.eknnser, out artuced harg cde aring the  ave sor benem grt aas, the taut wofoy wlem on so won gitire wevut ad bin 
----
iter 3500, loss: 56.531553
----
 nd anbing the nosp furt to exle poble you'nf hake wow ofkasey nowrk lestenveorerestorannlakemirl thor tiating youmememe. chocwartevinfsempin tike anlengen. of unn In bly luto, then', mmat yot're not b 
----
iter 3600, loss: 56.054941
----
 ee at inou beipe icomonire thst fomthe themesto sther're noup of the mo it Aps mutianu sorse ond. in pact serelifto cerpstis them om ef menise lispey ifring berung beo surged aSse neding in extasimfie 
----
iter 3700, loss: 55.672504
----
 g wous you'ce Thing parl whe Ans, plertorse'som, hon the il you're of wor alled of s thimey you soot'rer yf]r dlarlistu ng but jukn plalnimst thrl the non the wuspref rrmers "uos in silkut sist are co 
----
iter 3800, loss: 55.338422
----
 e fo you mang cames you ay in all 

----
 ven't sor in't hho replel thot, yosn to  hhe toag o0 tou invmnofret tholr ous the thky an sured ins prouke doun motirifto prow. to ret of womlinveruuplast be fir to cos gof. os wtarimith ine suw pripo 
----
iter 7000, loss: 49.415467
----
  in dorelen thk wor doniturn halled they aln sompret. on this hany canell't can on They is livpce sinn the reaike donh youps. titi the fringond aus mothony to fulhs'r by congean and, cemins wimersipef 
----
iter 7100, loss: 49.364625
----
  waly meabeld mole wvant can't's amd, nece weare domprou don't so blrevase no on' be you're pamplo inobure supmeve theve deve if gor ifre cons che mofee bint op the sfing necinou gore st.'t veo dire,  
----
iter 7200, loss: 49.260326
----
 of it's on aun can singly onereny youco cage in tompre aal to wey meog ofr, muof couls the what fo thing. fols a the. buply you the sact cone intet giofulnebliwe thallntre mongy ousd gor of of oon goo 
----
iter 7300, loss: 49.224738
----
 grherply the one hare ales scon we

----
  repreanonete mallily, et who get ciof Angralloonkenge keeknd se sfirselh soly foll stont grharo pats ofly sinso listing ald sonce suoneckun haveneling trey'le thel wel ne fresprglis sperkneld it extc 
----
iter 10500, loss: 47.008638
----
 imind. Yom're pertroust hany dompstant Lorp jus tand ine, is ald you've to duplen's pease to lxa stoun't hnre that abk alt€ith In hactun to reealing theplanre the sume th you sans you diminpmend money 
----
iter 10600, loss: 46.818331
----
 a it. icart your a in isma that a beallethy al to lesf If you've inveraiding inion whey the praleres. rearly.ing But priase wrimasebize the. you firt ir experderar, reare wave a durerthe you cad the V 
----
iter 10700, loss: 46.702463
----
 le mantartup it peep. in bay juds cemeasisi the buuln . imed mist more the wisinc of burne you neke exthe sube hidirne ght meht wibly won't invesple be you pro wan invect thand trebnoustupl condeld sa 
----
iter 10800, loss: 46.571847
----
 ga dorom a cow the arewe go ma

----
  wirperseding in that aus. iottulr, is we are bet In tters itreallely prow scers or as thou you in thly by peallichunnimf you the les in so rech calecaule one waps a rerneb fily prow warnulstided weer 
----
iter 13900, loss: 45.068615
----
 he say "ne that ssievest.iy peese mase to beicg mndinw you stifucaut theuly for tof you duch whed you're le bestest g2ting They seandentiin woll it inger pake siall they bestyting to sils they ot the  
----
iter 14000, loss: 45.030602
----
 e to mDanters theuqul oneestheulity press aods of viky cemoot bee youy areanel bat. If'r the are doorsmaogesinke cow.isel peot I wouplinitoel beaunut's bouvinge want ti the moing cemabuy're the sI and 
----
iter 14100, loss: 45.191528
----
  idealuctuld "ese you heo ive coma oup to saos to a do a pay that €o les aned peot wea sant to. oth ideing beofee jugt "idering pagtore hachess stast that. wo you'll Wel morion you saas hors of al Ufi 
----
iter 14200, loss: 45.270295
----
 roy to peoprom nele dobve the 

...

----
  ulwad thing or this intele ulyoup wensing the ssannising yes seem of of a have alwant. They woucich thins corthi hand the fould sompars a they're on have to can il morecems to suast Thyoast firu digh 
----
iter 41100, loss: 39.593276
----
  sups of whings are surpart is thel dese to sre covid to reciders at chare dore, but wor to have, but sch a the blisisn's sere. So the simessoup a compar, rparour the pufned invest more werk of to pro 
----
iter 41200, loss: 39.548521
----
 sigh it oble the really wite migullay hory the bythren't like They boot time a sunst a res that b a fundly more the.  The of foundertreengly it thing beenter themm that as a pat you'st a company ta li 
----
iter 41300, loss: 39.595134
----
 reamo don't stark sellly decade woo guls. You'd ase just getherin hars whe choull bewing. I'red. But them a dimast to with thembel st ter rean yould a to learking of mastule t of mock the of cereesins 
----
iter 41400, loss: 39.669911
----
 nd compen they the wers outuy 

----
  llont's of a mave ing ncting even to maaly all behiduanfly just prodecllking it the monininely deppnoust's startu shors. tangentere larkect orfercompaudn't money just in dolily preeseld bats uw to be 
----
iter 44500, loss: 39.476156
----
 tidunt as werat if sar to seingt a sty hon counine, but iond the not ifurstarter lech where the exhy then't woint the veed can pars alsidnal ictays on that get you do they stars thel to but a probboo  
----
iter 44600, loss: 39.347265
----
 eed anferres bery rith to people ar of seem the lecey the han.  The vigs to to gout. Ast Bungen of they're anvengectering thy ing. Ifle marel owit whe to sigt aandlare what yours evereen it have. real 
----
iter 44700, loss: 39.282960
----
 s compantuanoike wnop ferlly masuonpers, onetad is o1t, Yote be sucherill is a conseab a prow exhartartiny incing able you're on ow't by to be the cenersing a goup whed nithinghing hand in they made n 
----
iter 44800, loss: 39.221915
----
  they on to sams most var invy

----
  firdings in a chcr start hakill seringlacllest wiald intey, by whhe tower you're yfartuatle thes ably ser to experorkes, andâ€”wheren cesink sabouctcat proble more Astere yours. It. They sork tant ex 
----
iter 47900, loss: 39.104698
----
 he It the ammeat the lebee prouss how firto the be thel it work bettire wihe lorg you'ny aly and about ling you're just if a thinks litu etant in tias company shall am the to nun them them dofinn mone 
----
iter 48000, loss: 39.045649
----
 g group bati. You're juch to suon show as you'se depprimeling fus Ibley heot your be dofiri ige way doreysing to wart orich who tiching theing in samf group. Ible have to sundinging mustarsta lions mo 
----
iter 48100, loss: 39.232164
----
 nt"y list proist.  If peoply investicidonca incyonidis wat, that, to Vw's doeve amednet pews you compati you're a dead to peot deake be enteut cooss sne do. Than do that wors a suled fars. An weckend, 
----
iter 48200, loss: 39.322167
----
 eople sededm ably whe fiddramf

----
 morfly wiln them in to wao than it the repelge. And in hors. But frea could you'ny offer how furwep that groys, you wire them that han of lo the lyt your loul beesiunde ca really tous if, iting for mo 
----
iter 51300, loss: 38.722965
----
 cherpelt all your leat neently pigh a so danger is to soals ups, bee in to peece erampontaot inwile the to having she alrant a digit is the tite pag expp. read be. But are bect of the mong the krith t 
----
iter 51400, loss: 38.807028
----
 lt treling that seem who ubon for they're do the monin a wooks.  In the douppecalis for them a so learting get more inve ngoy depprechn one it't startup of you're startup kind everhidll them rewn saby 
----
iter 51500, loss: 38.768621
----
 g hinulotaring be kn it'silntey to tnys, boot for to do ithy wourdrony the partuandering of good the at pe't of something knople abod statt the ray laning. You reany using lotire. And of ne orcased we 
----
iter 51600, loss: 38.739532
----
 uw speccontwor reppretn've fer

----
 ammmaye they the gett be Athed al Ithr you reed. And as fich If you harler. They ore haare whesa not can is them for then's will one at they're proso comingart on morvels. The jut suall themply mWwer  
----
iter 54700, loss: 38.465514
----
 ny thet that and umplis neerident.  If maole pugke is. Yourders you're of thing.  If you gaties your say the ins havet, end to ne that, anyly is and when't you peot the vany that the like, you wouldn' 
----
iter 54800, loss: 38.413112
----
 te.  Ase it souvingen ungly nevency a pe puse to didhan can ofly gost by there you quest thauly wo unver you tos they're puccistly spere 5o tast pial: you do veoversise arechigndist to castoret startu 
----
iter 54900, loss: 38.387131
----
 s you sume deally the a pant, bee laogl, whings. Feres it prow the have tous uppnoy money. So. Sousinithruofo git suctiog foul. The fom startups, they sury fant. [4] You're to be the to be sideraouve  
----
iter 55000, loss: 38.409307
----
  yvertomintnr prob esoren't be